# <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Welcome to Rideshare AI Lakehouse powered by BigQuery Studio

Watch the demo on <a href="https://youtu.be/5ZStKD4joK4">YouTube</a>

## **Step 01: Overview**

### Summary


This portion of the demo will showcase how you can build an AI Lakehouse using Google Cloud's Data and Analytics stack.  The demo is for our fitcious company Rideshare Plus where we will use both qualitative and quantitative analysis to build a complete customer and driver profile.  This notebook will highlight:
- Using LLMs to extract customer and driver preferences based upon customer reviews.
- Using LLMs to understand our customers and to create a summarized customer understand to improve our interactions with our customers.
- Using LLMs to understand our drivers and how they are interacting with our customers and an overall customer statisification summary.

All code is located on GitHub: https://goo.gle/dagd

Please check out the other part of the demo which uses AI, image object analysis and data at scale to predict the most profitable pickup locations.

### Details

- For each customer review
  - Speech to Text

- Process the Customer Reviews (extract knowledge)
  - Sentiment
  - Extract Themes
    - Driver view
    - Customer view
  - Summarize Themes
    - For each Driver
    - For each Customer
  - Summarize All Reviews
    - For each Driver
      - We can use this as an Employee Review for the driver to suggust training.
    - For each Customer
      - We can use this to inform the driver of the customer preferences so the driver can taylor their ride for the customer.
  - Perform Quantitative analysis and summarize with LLM
     - For each Driver
     - For each Customer

- LLM (Other)
  - LLM can be used to see if our LLM Summary is:
    - Good Quality
    - Correct for puncation and grammer

## **Step 02: Customer Review Speech to Text**

### Summary


We will start by processing our audio files at scale. These can be achieved through object tables which will monitor a storage account for new customer reviews. Upon detecting a review, BigQuery can ask Vertex AI Speech to Text (STT) for the extracted text.

For each audio file containing the review we run it through the STT service to extract the text.

* First we create an Object Table pointing to the GCS bucket holding the audio files.
* We call `BQML.TRANSCRIBE audio`

Inspect the Object Table

In [ ]:
%%bigquery
SELECT REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') as trip_id,*
  FROM EXTERNAL_OBJECT_TRANSFORM(TABLE ${bigquery_rideshare_llm_raw_dataset}.biglake_rideshare_audios, ['SIGNED_URL'])
 LIMIT 5;

Process one particular customer audio review:
 - Get the signed_url using `EXTERNAL_OBJECT_TRANSFORM` on the Object Table
 - Download the audio file and inpect it (play it)
 - Run the speech to the cloud function and review the transcription results
 - Change the Trip Id to 24406991 for a Negative review

In [ ]:
trip_id = '30618713' #@param{type:"string"}
params = { "trip_id": trip_id}

In [ ]:
%%bigquery signed_url --params $params
WITH signed_urls AS (
  SELECT signed_url,
         REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') as trip_id
  FROM EXTERNAL_OBJECT_TRANSFORM(TABLE ${bigquery_rideshare_llm_raw_dataset}.biglake_rideshare_audios, ['SIGNED_URL']))
 SELECT signed_url FROM signed_urls WHERE trip_id = @trip_id;

In [ ]:
import requests
response = requests.get(signed_url['signed_url'][0],stream=True)
with open(f'text_synth_{trip_id}.mp3', 'wb') as f:
  f.write(response.content)

In [ ]:
import IPython.display
IPython.display.Audio(f'text_synth_{trip_id}.mp3',rate=16000)

In [ ]:
%%bigquery --params $params
BEGIN
CREATE TEMP TABLE customer_reviews_transcript  AS (SELECT ${bigquery_rideshare_llm_raw_dataset}.ext_udf_ai_extract_text(signed_url) AS customer_review_text,
       REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') as trip_id
  FROM EXTERNAL_OBJECT_TRANSFORM(TABLE ${bigquery_rideshare_llm_raw_dataset}.biglake_rideshare_audios, ['SIGNED_URL']) WHERE REGEXP_EXTRACT(uri,  r'text_synth_(\d+)\.mp3') = @trip_id );
SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
       prompt,
       ml_generate_text_result
  FROM ML.GENERATE_TEXT(MODEL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
      ( SELECT CONCAT('The following phase is a customer review for a rideshare company. ',
                      'Fix grammar and punctuation in the phase. ',
                      'Correct any words that might be out of context (e.g. ride chair should be rideshare). ', 
                      'Phrase: ', customer_review_text) as prompt
          FROM customer_reviews_transcript
         WHERE trip_id = @trip_id),
STRUCT(
  1  AS temperature,
  512 AS max_output_tokens,
  0  AS top_p,
  1  AS top_k
  ));
 END;

## **Step 03: Review Sentiment**

### Summary

For each customer review we run it through the LLM.
1. First we construct a prompt and save this in a field in our table.
2. Ask the LLM to score these in bulk.
3. We then extract the scored text from the JSON result.

*The complete code is in notebook: rideshare_llm_step_01_customer_sentiment_analysis*

### Code

In [ ]:
%%bigquery

-- Select a Positive Review from the Enriched Zone
-- For each review we created a LLM prompt (llm_sentiment_prompt)
  /* Sample Prompt:
  For the given review classify the sentiment as Positive, Neutral or Negative.
  Review: My rideshare driver was amazing! He was very professional and attentive.
          He made sure to pay close attention to the road without any distractions. He also made sure to provide me with a comfortable and safe ride.
          I would definitely recommend this driver to anyone looking for a rideshare.
  */
-- We will then use this prompt to pass to our LLM and receive the scored output

SELECT customer.customer_name,
       driver.driver_name,
       customer_review.customer_review_text,
       customer_review.llm_sentiment_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
               ON customer_review.customer_id = customer.customer_id
       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
               ON customer_review.driver_id = driver.driver_id
 WHERE trip_id = 20857125;

**ML.GENERATE_TEXT model**

We will use the ML.GENERATE_TEXT model which is Google's PaLM 2 Large Language Model (LLM)
- <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text">Documentation Link</a>

In [ ]:
%%bigquery

-- Same query as above, but we will pass the prompt to the LLM along with setting our LLM model parameters
SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
       prompt,
       ml_generate_text_result
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
      (SELECT customer_review.llm_sentiment_prompt AS prompt
         FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
              INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
                      ON customer_review.customer_id = customer.customer_id
              INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
                      ON customer_review.driver_id = driver.driver_id
       WHERE trip_id = 20857125),
STRUCT(
  0  AS temperature,
  50 AS max_output_tokens,
  0  AS top_p,
  1  AS top_k
  ));

**Understanding our LLM parameters**
- <a href="https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text#request_body">Documentation Link</a>

|Parameter|Range|Description|
|----|----|----|
|temperature|0.0 -> 1.0|Temperature controls the degree of randomness in token selection. Lower temperatures are good for prompts that require a less open-ended or creative response, while higher temperatures can lead to more diverse or creative results. A temperature of 0 means that the highest probability tokens are always selected. In this case, responses for a given prompt are mostly deterministic, but a small amount of variation is still possible.|
|max_output_tokens|1 -> 1024|Sets the maximum number of tokens that the model outputs. Specify a lower value for shorter responses and a higher value for longer responses. The default is 50.  A token is approximately four characters.|
|top_p|0.0 -> 1.0|Specify a lower value for less random responses and a higher value for more random responses.  This value sets a threshold probability and selects the smallest set of tokens whose cumulative probability exceeds the threshold.|
|top_k|1 -> 40|Specify a lower value for less random responses and a higher value for more random responses.  A `top_k` of 1 means the selected token is the most probable among all tokens in the model's vocabulary (also called greedy decoding). In contrast, a `top_k` of 3 means that the next token is selected from the top 3 most probable tokens (using temperature). For each token selection step, the `top_k` tokens with the highest probabilities are sampled. Then tokens are further filtered based on `top_p` with the final token selected using temperature sampling.|


## **Step 04: Extract Themes from Customer Reviews**

### Summary

- We want to use our customer reviews to understand our customer preferences.  - At the same time we can use this information to understand what our drivers are doing.  
- For instance if customers consistently complain about a driver's car being too cold, we can extract that the driver keeps their car cold.  
- If the same customer mentions that they are typically cold we can extract that the customer likes a warmer car.

*The complete code is in notebook: rideshare_llm_step_02_driver_themes and rideshare_llm_step_03_customer_themes*

### Code (Driver Theme Extraction)

Create a **Qualitative** data analysis for **Driver** Habits
  - We have found some patterns that we want to dive deeper into:
    - trunk space
    - driving speed
    - hours worked
    - preferred pickup locations
    - average trip distance
    - crossing state lines
    - vechicle cleanliness
    - vechile temperature
    - maximum passengers
    - conversation with customer
    - music playing
    - distracted driver
    - target pay

*The complete code is in notebook: rideshare_llm_step_02_driver_themes*    

In [ ]:
%%bigquery

-- View the customer review

SELECT customer.customer_name,
       driver.driver_name,
       customer_review.customer_review_text,
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
               ON customer_review.customer_id = customer.customer_id
       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
               ON customer_review.driver_id = driver.driver_id
 WHERE trip_id = 15369855;

In [ ]:
%%bigquery

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
       prompt,
       ml_generate_text_result
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT CONCAT(
"""
Classify the text as one or more of the following categories and return in the below json format.
- "trunk space small"
- "trunk space large"
- "driving too fast"
- "driving too slow"
- "clean car"
- "dirty car"
- "car too hot"
- "car too cold"
- "driver likes conversation"
- "driver likes no conversation"
- "driver likes music"
- "driver likes no music"
- "distracted driver"

JSON format: [ "value" ] 
Sample JSON Response: [ "dirty car", "car too cold" ] 


Text:
""", customer_review_text) AS prompt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
         WHERE trip_id = 15369855),
STRUCT(
  0  AS temperature,
  1024 AS max_output_tokens,
  0  AS top_p,
  1  AS top_k
  ))

### Code (Customer Theme Extraction)

Create a **Qualitative** data analysis for **Customer** Habits
  - We have found some patterns that we want to dive deeper into:
    - trunk space
    - driving speed
    - vechicle cleanliness
    - vechile temperature
    - conversation
    - music playing
    - distracted driver

*The complete code is in notebook: rideshare_llm_step_03_customer_themes*


In [ ]:
%%bigquery

-- Same review as the driver, but now from the customers standpoint of view

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
       prompt,
       ml_generate_text_result
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT CONCAT(
"""
Classify the customer review as one or more of the following categories and return in the below json format.
- "customer has small luggage"
- "customer has large luggage"
- "customer likes to drive fast"
- "customer likes to drive slow"
- "customer likes a clean car"
- "customer likes the temperature warm"
- "customer likes the temperature cold"
- "customer likes conversation"
- "customer likes no conversation"
- "customer likes music"
- "customer likes quiet"

JSON format: [ "value" ] 
Sample JSON Response: [ "customer likes music", "customer likes a clean car" ] 

Review:
""", customer_review_text) AS prompt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
         WHERE trip_id = 15369855),
STRUCT(
  0  AS temperature,
  1024 AS max_output_tokens,
  0  AS top_p,
  1  AS top_k
  ))

## **Step 05: Summarize Themes**

### Summary

Now that we have extracted all the themes for both the driver and customer poiint of view:
- Each theme needs to mapped to a category (e.g. music)
  - We might have 10 "likes music on"
  - We might have 50 "likes music off"
- For each category (music) determine the prevailing preference
  - Since we have 50 "likes music off" we will use that as our "winner"
- Write a summary using the LLM to process this data so we can review in just a few sentences.

*The complete code is in notebook: rideshare_llm_step_04_driver_summary and rideshare_llm_step_05_customer_summary*

### Code (Driver Theme Summary)

Process the themes for each driver by passing their attibutes to the LLM.

*The complete code is in notebook: rideshare_llm_step_04_driver_summary*

In [ ]:
%%bigquery

-- After we have determined the prevailing attribute for our Driver, combine them so we can create a LLM Prompt

SELECT driver_id,
       STRING_AGG(
       CASE WHEN TRIM(extracted_driver_attribute) = 'driver likes music'           THEN 'Attribute: the driver likes the radio on\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no music'        THEN 'Attribute: the driver likes the radio off\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space large'            THEN 'Attribute: the driver has a large amount of trunk space\n'
            WHEN TRIM(extracted_driver_attribute) = 'trunk space small'            THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'Category: trunk space small'  THEN 'Attribute: the driver has a small trunk\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes conversation'    THEN 'Attribute: the driver likes to have a conversation with the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driver likes no conversation' THEN 'Attribute: the driver does not like to talk to the passengers\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too fast'             THEN 'Attribute: the driver tends to drive too fast\n'
            WHEN TRIM(extracted_driver_attribute) = 'driving too slow'             THEN 'Attribute: the driver drives too slow\n'
            WHEN TRIM(extracted_driver_attribute) = 'clean car'                    THEN 'Attribute: the driver keeps their car clean on the inside and out\n'
            WHEN TRIM(extracted_driver_attribute) = 'dirty car'                    THEN 'Attribute: the driver has a dirty car\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too hot'                  THEN 'Attribute: the driver has a keeps their car too warm inside\n'
            WHEN TRIM(extracted_driver_attribute) = 'car too cold'                 THEN 'Attribute: the driver keeps their car too cold\n'
            WHEN TRIM(extracted_driver_attribute) = 'distracted driver'            THEN 'Attribute: the driver is a distracted driver\n'
            WHEN TRIM(extracted_driver_attribute) = 'safe driver'                  THEN 'Attribute: the driver is a safe driver\n'
            ELSE ''
       END,'') AS driver_attribute_agg
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver_attribute`
GROUP BY driver_id
LIMIT 5;

Create the Driver Attibute Summary

In [ ]:
%%bigquery

-- We will take the concatenated list of attributes and run them through our LLM to create a Driver Theme/Attibute Summary

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT
"""
Write a 20 to 500 word summary for the following attributes.
1. Randomly sort the attributes
2. Select the first 3 to 5 attributes
3. Write the summary in present tense for only the first 3 to 5 attributes
4. The driver's name is Danilo Gottardi
5. Use sentences with varying lengths
6. Write 2 to 5 sentences

Attribute: the driver likes the radio off
Attribute: the driver has a small trunk
Attribute: the driver does not like to talk to the passengers
Attribute: the driver drives too slow
Attribute: the driver has a dirty car
Attribute: the driver keeps their car too cold
Attribute: the driver is a distracted driver
""" AS prompt),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  .5   AS temperature,  -- Change this to Zero and One to see the results (demo default is .5)
  1024 AS max_output_tokens,
  1    AS top_p,
  40   AS top_k
  ));

**Demo:** Rerun the above SQL changing the temperature parameter to see the varying responses.

### Code (Customer Theme Summary)

Process the themes for each customer by passing their attibutes to the LLM.

*The complete code is in notebook: rideshare_llm_step_05_customer_summary*

In [ ]:
%%bigquery

-- View our aggregated customer preferences

SELECT customer_id,
       STRING_AGG(extracted_customer_attribute,', ') AS customer_attribute_agg
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_attribute`
WHERE customer_id IN (3760, 549, 2672, 3025)
GROUP BY customer_id;

Create the Customer Attibute Summary

In [ ]:
%%bigquery

-- We will take the concatenated list of attributes and run them through our LLM to create a Customer Theme/Attibute Summary
-- Customer Id: 3760

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT
"""
Write a 50 to 600 word summary for the following customer preferences.
1. The customer's name is Paulina Peruzzi.
2. Write the summary in present tense.
3. Write the summary from the customers prespective.
4. Do not repeat the same subject in the summary.
5. Write 3 to 6 sentences.

Preference: likes a clean car.
Preference: likes a warm vehicle inside.
Preference: likes to have a conversation.
Preference: likes their driver to drive slow.
Preference: prefers the radio off.
""" AS prompt),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  .75  AS temperature,  -- Change this to Zero and One to see the results (demo default is .75)
  1024 AS max_output_tokens,
  1    AS top_p,
  40   AS top_k
  ));

**Demo:** Rerun the above SQL changing the temperature parameter to see the varying responses.

## **Step 06: Summarize All Reviews**

### Summary

For each Driver summarize all the customer reviews for that driver.

For each Customer summarize all their reviews for their various drivers.

*The complete code is in notebook: rideshare_llm_step_04_driver_summary and rideshare_llm_step_05_customer_summary*

### Code (Driver Review Summary)

In [ ]:
%%bigquery

SELECT CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
              '1. Write the summary in present tense.\n',
              '2. Write the summary from the customers prespective.\n',
              '3. Do not repeat the same subject in the summary.\n',
              '4. The reviews are for the driver ', driver_name, ' at a rideshare company.\n',
              '5. The reviews are written by different customers.\n',
              '6. Write 3 to 6 sentences.\n',
               customer_review_agg) AS prompt
  FROM (
    SELECT customer_review.driver_id,
               STRING_AGG(CONCAT("Review: ",customer_review_text),'\n') AS customer_review_agg
          FROM (SELECT driver_id,
                       customer_review_text
                  FROM (SELECT trip.driver_id,
                               trip.pickup_time,
                               customer_review.customer_review_text,
                               ROW_NUMBER() OVER (PARTITION BY trip.driver_id ORDER BY trip.pickup_time) AS row_nbr
                          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
                               INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.trip` AS trip
                                       ON customer_review.trip_id = trip.trip_id
                                      AND customer_review.driver_id = 23 -- Change this for demoing different drivers
                       ) AS reviews
                 WHERE row_nbr < 50 -- top 50 most recent
               ) AS customer_review
        GROUP BY driver_id

        ) AS top_reviews
        INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
                ON driver.driver_id = top_reviews.driver_id;

In [ ]:
%%bigquery

-- Driver

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (
        SELECT CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
                      '1. Write the summary in present tense.\n',
                      '2. Write the summary from the customers prespective.\n',
                      '3. Do not repeat the same subject in the summary.\n',
                      '4. The reviews are for the driver ', driver_name, ' at a rideshare company.\n',
                      '5. The reviews are written by different customers.\n',
                      '6. Write 3 to 6 sentences.\n',
                       customer_review_agg) AS prompt
          FROM (
            SELECT customer_review.driver_id,
                       STRING_AGG(CONCAT("Review: ",customer_review_text),'\n') AS customer_review_agg
                  FROM (SELECT driver_id,
                               customer_review_text
                          FROM (SELECT trip.driver_id,
                                       trip.pickup_time,
                                       customer_review.customer_review_text,
                                       ROW_NUMBER() OVER (PARTITION BY trip.driver_id ORDER BY trip.pickup_time) AS row_nbr
                                  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review` AS customer_review
                                       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.trip` AS trip
                                               ON customer_review.trip_id = trip.trip_id
                                              AND customer_review.driver_id = 23 -- Change this for demoing different drivers
                               ) AS reviews
                         WHERE row_nbr < 50 -- top 50 most recent
                       ) AS customer_review
                GROUP BY driver_id
                ) AS top_reviews
                INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
                        ON driver.driver_id = top_reviews.driver_id),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  .75  AS temperature,  -- Change this to Zero and One to see the results (demo default is .75)
  1024 AS max_output_tokens,
  1    AS top_p,
  40   AS top_k
  ));

### Code (Customer Review Summary)

In [ ]:
%%bigquery

SELECT CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
              '1. The reviews are written by ', customer.customer_name, '.\n',
              '2. Write the summary in present tense.\n',
              '3. Do not repeat the same subject in the summary.\n',
              '4. The reviews are for different drivers.\n',
              '5. The reviews are a single rideshare company.\n',
              '6. The drivers all work for the rideshare company.\n',
              '7. Write 3 to 6 sentences.\n',
               customer_review_agg)
  FROM (SELECT customer_id,
               STRING_AGG(CONCAT('Review: ',customer_review_text,'\n'),'') AS customer_review_agg
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
         WHERE customer_id = 3556
         GROUP BY customer_id) AS customer_review
       INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
               ON customer.customer_id = customer_review.customer_id;

In [ ]:
%%bigquery

-- Customer

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
                      '1. The reviews are written by ', customer.customer_name, '.\n',
                      '2. Write the summary in present tense.\n',
                      '3. Do not repeat the same subject in the summary.\n',
                      '4. The reviews are for different drivers.\n',
                      '5. The reviews are a single rideshare company.\n',
                      '6. The drivers all work for the rideshare company.\n',
                      '7. Write 3 to 6 sentences.\n',
                      customer_review_agg) AS prompt
         FROM (SELECT customer_id,
                      STRING_AGG(CONCAT('Review: ',customer_review_text,'\n'),'') AS customer_review_agg
                 FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
                WHERE customer_id = 3556
                GROUP BY customer_id) AS customer_review
              INNER JOIN `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
                      ON customer.customer_id = customer_review.customer_id),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  1    AS temperature,  -- Change this to 0, .75 and 1 to see the results (demo default is 1)
  1024 AS max_output_tokens,
  1    AS top_p,
  40   AS top_k
  ));

## **Step 07: Perform Quantitative analysis and summarize with LLM**

### Summary

Using a our trip data, it was summed, averaged and analyized to find some common patterns of drivers.  The data is then passed to the LLM to create a summary.

To see how the quanitative data was created please see the stored procedure
- ${project_id}.${bigquery_rideshare_llm_enriched_dataset}.sp_step_01_quantitative_analysis

*The complete code is in notebook: rideshare_llm_step_06_driver_quantitative_analysis and rideshare_llm_step_07_customer_quantitative_analysis*

### Code (Driver Quantitative Analysis)

In [ ]:
%%bigquery
SELECT driver_quantitative_analysis_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.driver` AS driver
 WHERE driver_id = 66;

In [ ]:
%%bigquery

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT
"""
Write a 3 to 8 sentence summary of the following attributes of a driver in third person gender neutral form:
- The driver picks up customers at 4 pickup locations. This is an average number. These locations are: Midtown South, Elmhurst, University Heights/Morris Heights, Brownsville.
- The driver is not willing to drive accross state lines.
- The driver typically does not pickup or dropoff at the airport.
- The driver only works on weekdays.
- The driver likes to work a split shift which appears to target rush hour.
""" AS prompt),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  .8   AS temperature,  -- Change this to Zero and One to see the results (demo default is .8)
  1024 AS max_output_tokens,
  .70  AS top_p,
  25   AS top_k
  ));

### Code (Customer Quantitative Analysis)

In [ ]:
%%bigquery

SELECT customer_quantitative_analysis_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
 WHERE customer_id  = 3219;

In [ ]:
%%bigquery

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT
"""
Write a 2 to 3 sentence summary of the following attributes of a customer who uses a rideshare services.
- Martin Hernandez uses the service on weekdays.
- Martin Hernandez likes to use the service during the morning and afternoon rush hours.
""" AS prompt),
STRUCT(
  -- Set the parameters for a more creative/random responses
  -- You can adjust these during the demo
  .8   AS temperature,  -- Change this to Zero and One to see the results (demo default is .8)
  1024 AS max_output_tokens,
  .70  AS top_p,
  25   AS top_k
  ));

## **Step 08: Using LLMs to correct LLMs**

### Summary

During the demo sometime the LLM output incomplete text and had grammatical errors.  You can use the LLM to correct this.

### Code

In [ ]:
%%bigquery

SELECT JSON_VALUE(ml_generate_text_result, '$.predictions[0].content') AS result,
  FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_curated_dataset}.cloud_ai_llm_v1`,
       (SELECT
"""
For the following text only make the following changes:
1. Correct spelling
2. Correct incomplete sentences
3. Fix punctuation mistakes
4. Use the correct pronouns of he/him

Text: The drivers car was clean and has a large trunk for storage..  Mike driving very carefully.  He like to have radio ons.  They is not distracted while driving
""" AS prompt),
STRUCT(
  .1   AS temperature,
  1024 AS max_output_tokens,
  .1  AS top_p,
  10   AS top_k
  ));